# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

Задание делали студенты группы ИАД-4 Ваниев Константин, Максименко Захар, Максимовская Анастасия

### 1. Сбор данных [3 балла]

Мы выбрали город Тикси: https://www.gismeteo.ru/weather-tiksi-3897/10-days/

Ниже приводим код для запуска паучка scrapy, который сохраняется в файле .py. 

In [17]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "tiksi"
    start_urls = [
        'https://www.gismeteo.ru/weather-tiksi-3897/10-days/'
    ]

    def parse(self, response):
        table = response.css("div.widget__body")[0]
        for i in range(10):
            yield {
                'день': table.css("div.w_date__day::text")[i].extract(),
                'число' : table.css("span.w_date__date.weekend::text, span.w_date__date.black::text")[i].extract(),
                'минимальная температура': table.css("div.mint::text")[i].extract(),
                'максимальная температура': table.css("div.maxt::text")[i].extract(),
                'скорость ветра': table.css("div.w_wind__warning.w_wind__warning_::text")[i].extract(),
                'уровень осадков': table.css("div.w_precipitation__value::text")[i].extract(),
            }
            



In [95]:
import pandas as pd

df = pd.read_json("tiksi.json")
df.to_csv("tiksi.csv", encoding = 'utf-8')

In [96]:
df = pd.read_csv("tiksi.csv")
for i in range(len(df['уровень осадков'])):
    df['уровень осадков'][i] = df['уровень осадков'][i].strip()
    df['число'][i] = df['число'][i].strip()
df.drop('Unnamed: 0', axis = 1, inplace = True)
df

C:\Users\kostawan\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kostawan\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,день,максимальная температура,минимальная температура,скорость ветра,уровень осадков,число
0,Вс,−22,−27,5,"0,7",18 фев
1,Пн,−22,−27,12,1,19
2,Вт,−20,−23,19,"2,5",20
3,Ср,−20,−24,15,"1,7",21
4,Чт,−23,−24,13,"1,6",22
5,Пт,−24,−27,9,"0,4",23
6,Сб,−22,−27,8,"0,4",24
7,Вс,−20,−23,11,"0,8",25
8,Пн,−24,−28,14,"0,4",26


In [97]:
df = df.filter(['максимальная температура', 'минимальная температура', 'скорость ветра', "уровень осадков"])
df = df.transpose()
df.columns = ['18.02 (пт)', '19.02 (сб)', '20.02 (вс)', '21.02 (пн)', '22.02 (вт)', '23.02 (ср)', '24.02 (чт)', '25.02 (пт)', '26.02 (сб)']
df

,18.02 (пт),19.02 (сб),20.02 (вс),21.02 (пн),22.02 (вт),23.02 (ср),24.02 (чт),25.02 (пт),26.02 (сб)
максимальная температура,−22,−22,−20,−20,−23,−24,−22,−20,−24
минимальная температура,−27,−27,−23,−24,−24,−27,−27,−23,−28
скорость ветра,5,12,19,15,13,9,8,11,14
уровень осадков,"0,7",1,"2,5","1,7","1,6","0,4","0,4","0,8","0,4"


### 2. Генератор описания прогноза погоды [4 балла]

In [98]:
import pandas as pd
import numpy as np
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [99]:
for i in range(len(df.columns)):
    for j in range(len(df)):
        if type(df.iloc[j][i]) == str:
            df.iloc[j][i] = df.iloc[j][i].replace(',', '.')
            df.iloc[j][i] = df.iloc[j][i].replace('−', '-')
            df.iloc[j][i] = float(df.iloc[j][i])

In [100]:
df

,18.02 (пт),19.02 (сб),20.02 (вс),21.02 (пн),22.02 (вт),23.02 (ср),24.02 (чт),25.02 (пт),26.02 (сб)
максимальная температура,-22,-22,-20,-20,-23,-24,-22,-20,-24
минимальная температура,-27,-27,-23,-24,-24,-27,-27,-23,-28
скорость ветра,5,12,19,15,13,9,8,11,14
уровень осадков,0.7,1,2.5,1.7,1.6,0.4,0.4,0.8,0.4


In [101]:
import re

In [102]:
def temp(city,day_1,day_2):
    parsed_city = morph.parse(city)[0].inflect({'loct'})
    parsed_day_1 = morph.parse(day_1)[0].inflect({'accs'})
    parsed_day_2 = morph.parse(day_2)[0].inflect({'ablt'})
    
    day1 = re.sub('[а,е,и,о,у,э,я,ю]', '', day_1)[:2]  # Чтобы получить сокращенные варианты дней недели
    day2 = re.sub('[а,е,и,о,у,э,я,ю]', '', day_2)[:2]
    
    for i in range (len(df.columns.str.find(day2))):
        if df.columns.str.find(day2)[i]>0:
            k=i
            break
        else:
            i+=1
    
    for i in range (len(df.columns.str.find(day1))):
        if df.columns.str.find(day1)[i]>0 and i>k:
            p=i
            break
        else:
            i+=1
    
    dtemp = (df.iloc[0,p]+df.iloc[1,p])/2 - (df.iloc[0,k]+df.iloc[1,k])/2 # за разницу температур я беру разницу между средними 
                                                                          # температурами в этот день
    if dtemp>0:
        verb='потеплело'
    else:
        verb='похолодало'
        
    if day_2=='вторник' or day_2=='среда':
        prep='со'
    else:
        prep='с'
        
    s = ' '.join(['В', parsed_day_1.word, 'в', parsed_city.word.title(), verb, 'на', str(abs(dtemp)), 
                  morph.parse("градус")[0].make_agree_with_number(int(dtemp)).word, 'по сравнению', prep, parsed_day_2.word, '.'])
    return s

In [103]:
temp('Тикси', 'суббота', 'четверг')

'В субботу в Тикси похолодало на 1.5 градусов по сравнению с четвергом .'

In [104]:
def wind(city,day_1,day_2):
    parsed_day_1 = morph.parse(day_1)[0].inflect({'accs'})
    parsed_day_2 = morph.parse(day_2)[0].inflect({'ablt'})
    
    day1 = re.sub('[а,е,и,о,у,э,я,ю]', '', day_1)[:2]     # Чтобы получить сокращенные варианты дней недели
    day2 = re.sub('[а,е,и,о,у,э,я,ю]', '', day_2)[:2]
    
    for i in range (len(df.columns.str.find(day2))):
        if df.columns.str.find(day2)[i]>0:
            k=i
            break
        else:
            i+=1
    
    for i in range (len(df.columns.str.find(day1))):
        if df.columns.str.find(day1)[i]>0 and i>k:
            p=i
            break
        else:
            i+=1
    
    dwind = df.iloc[2,p] - df.iloc[2,k]
    if dwind>0:
        verb='увеличится'
    else:
        verb='уменьшится'
        
    if day_2=='вторник' or day_2=='среда':
        prep='со'
    else:
        prep='с'
        
    s = ' '.join(['Скорость ветра', verb, 'на', str(abs(int(dwind))),
                  morph.parse("единица")[0].make_agree_with_number(abs(int(dwind))).word, 'в', parsed_day_1.word, 
                  'по сравнению', prep, parsed_day_2.word, '.'])
    return s

In [105]:
wind('Тикси', 'суббота', 'вторник')

'Скорость ветра увеличится на 1 единица в субботу по сравнению со вторником .'

In [106]:
def prec(n):
    dprec=df.iloc[3,n]-df.iloc[3,0]
    if dprec>0:
        verb='повысится'
    else:
        verb='понизится'
    s = ' '.join(['Уровень осадков', verb, 'на', str(abs(dprec)), 
                  morph.parse("единица")[0].make_agree_with_number(dprec).word,
                  'за', str(n), morph.parse("день")[0].make_agree_with_number(n).word,'.'])
    return s

In [107]:
prec(5)

'Уровень осадков понизится на 0.29999999999999993 единиц за 5 дней .'

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

###### Вопрос 1:

Генерацию текста по шаблонам можно использовать, например, при массовой рассылке писем, где основное тело письма может быть одним и тем же, а имена и малая часть письма (например, результаты тестирования/собеседования) меняться в зависимости от получателя письма. Кроме того, генерацию по шаблонам можно использовать в агрегаторах отзывов/комментариев к какому-то товару: в шаблон будут вставляться какие-то отличительные особенности товара, которые отметили покупатели, его достоинства и недостатки. 
Генерация текста об изменении числовых показателей может быть удобно в утренних новостях об изменении курса валюты за ночь, резком изменении стоимости акции (не просто колебания, а значительное изменение показателя), ключевой ставки и прочих важных макроэкономических показателей. 

###### Вопрос 2:

Можно вместо фиксированных шаблонов сделать шаблоны с рядом слов-синонимов, которые будут случайно/по правилу меняться, создавая ощущение уникальности каждого нового текста. Кроме того, можно менять порядок слов в тексте в такой степени, в которой это позволяет сделать гибкость русского языка. Однако, нужно следить за тем, чтобы текст оставался осмысленным, в чем и поможет знание синтаксической структуры текста.

Кроме того, знание синтаксической структуры может позволить отображать усиливающие оттенки, увеличивающие "естественность" текста. Например, мы бы могли в этом задании вставлять не числовые характеристики погоды, а оценочные слова "очень", "немного", "небольшой", которые могут сделать текст натуральнее (например, "В субботу будут небольшие осадки", "В понедельник будет жаркое утро")
